<!-- WARNING: THIS FILE WAS AUTOGENERATED! DO NOT EDIT! -->

# Application programming interface (API)

The following class defines the API of findmycells, which represents the intended way of how users interact with and use findmycells, if they are not using the graphical user interface (GUI; defined below).

In [1]:
#| echo: false
#| output: asis
show_doc(API)

---

[source](https://github.com/Defense-Circuits-Lab/findmycells/blob/main/findmycells/interfaces.py#L33){target="_blank" style="float:right; font-size:smaller"}

### API

>      API (project_root_dir:pathlib.PosixPath)

Initialize self.  See help(type(self)) for accurate signature.

# Graphical user interface

The following classes are used to create the graphical user interface of findmycells. Please note that classes will be listed here in an inverted hirarchy, such that you can find the main [`GUI`](https://Defense-Circuits-Lab.github.io/findmycells/API documentation/interfaces.html#gui) class at the very end, and classes that handle much more specific details at the beginning.

In [ ]:
class StrategyConfigurator:
    
    """
    This class implements the interface that let´s the user choose and 
    configurate the processing strategies. It will be placed inside of
    an accordion that is implemented in the `ProcessingStepPage`.
    It gets a list of all available processing strategies from the parent
    `ProcessingStepPage` and, thus, eventually from the `API` that checks
    for all available processing strategies in the corresponding processing
    submodule (e.g. "findmycells.preprocessing.strategies"). Upon initializing
    it´s dropdown widget, which let´s the user browser through the different
    available strategies, it also initializes an object of each strategy.
    This object can then be used to run it´s associated method 
    ".initialize_gui_configs_and_widget()" to build the specified widget, 
    using its `GUIConfigs` instance. Essentially, this contains a
    description of what the processing strategy does and, if applicable,
    widgets to specify all parameters that can be configurated for this 
    strategy. Finally, a "confirm & export" and a "remove" button allow
    the user to load or delete the current to or from the findmycells
    project, respectively.
    """
    
    def __init__(self,
                 available_strategy_classes: List,
                 parent_accordion: w.Accordion,
                 target_for_configs_export: List) -> None:
        self.available_strategy_classes = available_strategy_classes
        self.parent_accordion = parent_accordion
        self.target_for_configs_export = target_for_configs_export
        self.widget = self._initialize_widget()
        self._link_widgets_with_eventhandlers()

        
    def _initialize_widget(self) -> WidgetType:
        info_text = w.HTML(value = ('Please select one of the available processing methods from the dropdown menu below. '
                                    'Feel free to click through all listed methods, as each of them will display a '
                                    'short description of what exactly to expect and may also prompt you with some '
                                    'customization options. To select a method with all selected customization options, '
                                    'click on the "confirm selection & export configurations". Using the "remove method" '
                                    'button allows you to remove previously loaded methods again.'))
        self.dropdown = self._initialize_dropdown()
        self.confirm_and_export_button = w.Button(description = 'confirm selection & export configurations', layout = {'width': '30%'})
        self.remove_button = w.Button(description = 'remove method', layout = {'width': '20%'}, disabled = True)
        self.displayed_strat_widget = w.VBox([self.dropdown.value.widget], layout = {'width': '95%'})
        widget = w.VBox([info_text,
                         GUI_SPACER,
                         w.HBox([self.dropdown, self.confirm_and_export_button, self.remove_button]),
                         GUI_SPACER,
                         self.displayed_strat_widget])
        return widget
        
        
    def _link_widgets_with_eventhandlers(self) -> None:
        self.confirm_and_export_button.on_click(self._confirm_and_export_button_clicked)
        self.remove_button.on_click(self._remove_button_clicked)
        self.dropdown.observe(self._dropdown_option_changed, names = 'value')
        
    
    def _initialize_dropdown(self) -> WidgetType:
        dropdown_option_tuples = []
        for strategy_class in self.available_strategy_classes:
            strategy_obj = strategy_class()
            strategy_obj.initialize_gui_configs_and_widget()
            dropdown_option_tuples.append((strategy_obj.dropdown_option_value_for_gui, strategy_obj))
        return w.Dropdown(options = dropdown_option_tuples, layout = {'width': '50%'}) 
        
        
    def _get_own_position_idx_in_parent_accordion(self) -> int:
        return self.parent_accordion.children.index(self.widget)
        
        
    def _confirm_and_export_button_clicked(self, b) -> None:
        self._export_configs()
        self._change_disable_settings_of_customizable_widgets(disable_customizable_widgets = True)
        self._add_new_strategy_configurator_to_parent_accordion()
        self._change_own_accordion_tab_title(title = self.dropdown.value.dropdown_option_value_for_gui)
        self.parent_accordion.selected_index = None
        
        
    def _add_new_strategy_configurator_to_parent_accordion(self) -> None:
        new_strategy_configurator = StrategyConfigurator(available_strategy_classes = self.available_strategy_classes,
                                                         parent_accordion = self.parent_accordion,
                                                         target_for_configs_export = self.target_for_configs_export)
        self.parent_accordion.children = self.parent_accordion.children + (new_strategy_configurator.widget, )
        position_idx = len(self.parent_accordion.children) - 1
        self.parent_accordion.set_title(position_idx, 'Expand me to add a processing method')
                                                         
                                                         
    def _change_own_accordion_tab_title(self, title: str) -> None:
        position_idx = self._get_own_position_idx_in_parent_accordion()
        self.parent_accordion.set_title(position_idx, title)

        
    def _export_configs(self) -> None:
        selected_strategy_obj = self.dropdown.value
        current_configs = selected_strategy_obj.gui_configs.export_current_config_values()
        position_idx = self._get_own_position_idx_in_parent_accordion()
        self.target_for_configs_export.insert(position_idx, (selected_strategy_obj.__class__, current_configs))      
        
        
    def _change_disable_settings_of_customizable_widgets(self, disable_customizable_widgets: bool) -> None:
        self.remove_button.disabled = not disable_customizable_widgets
        self.dropdown.disabled = disable_customizable_widgets
        self.confirm_and_export_button.disabled = disable_customizable_widgets
        for widget in self.displayed_strat_widget.children[0].children:
            if hasattr(widget, 'disabled'):
                widget.disabled = disable_customizable_widgets
            elif type(widget) == w.HBox:
                if hasattr(widget.children[0], 'disabled'):
                    widget.children[0].disabled = disable_customizable_widgets
        
        
    def _remove_button_clicked(self, b) -> None:
        self._remove_configs()
        if len(self.parent_accordion.children) == 1:
            self._change_disable_settings_of_customizable_widgets(disable_customizable_widgets = False)
            self._change_own_accordion_tab_title(title = 'Expand me to add a processing method')
        else:
            currently_present_accordion_tabs = len(self.parent_accordion.children)
            currently_confirmed_strategies = len(self.target_for_configs_export)
            tabs_available_for_selection = currently_present_accordion_tabs - currently_confirmed_strategies
            if tabs_available_for_selection > 1:
                self._remove_own_tab_from_parent_accordion()
            else:
                self._change_disable_settings_of_customizable_widgets(disable_customizable_widgets = False)
                self._change_own_accordion_tab_title(title = 'Expand me to add a processing method')
                
                
    def _remove_own_tab_from_parent_accordion(self) -> None:
        tmp_children = list(self.parent_accordion.children)
        tmp_titles = []
        for idx in range(len(tmp_children)):
            tmp_titles.append(self.parent_accordion.get_title(idx))
        position_idx = self._get_own_position_idx_in_parent_accordion()
        tmp_children.pop(position_idx)
        tmp_titles.pop(position_idx)
        self.parent_accordion.children = tuple(tmp_children)
        for idx, title in enumerate(tmp_titles):
            self.parent_accordion.set_title(idx, title)
            
            
    def _remove_configs(self) -> None:
        position_idx = self._get_own_position_idx_in_parent_accordion()
        self.target_for_configs_export.pop(position_idx)
                
                
    def _dropdown_option_changed(self, change) -> None:
        new_selection = change.new
        self.displayed_strat_widget.children = (new_selection.widget, )

In [2]:
#| echo: false
#| output: asis
show_doc(PageButtonBundle)

---

[source](https://github.com/Defense-Circuits-Lab/findmycells/blob/main/findmycells/interfaces.py#L435){target="_blank" style="float:right; font-size:smaller"}

### PageButtonBundle

>      PageButtonBundle (bundle_id:str,
>                        page_screen:traitlets.traitlets.MetaHasTraits,
>                        all_navigator_buttons:List, api:__main__.API)

Helper class that provides a standard way to create an ABC using
inheritance.

In [3]:
#| echo: false
#| output: asis
show_doc(SettingsPage)

---

[source](https://github.com/Defense-Circuits-Lab/findmycells/blob/main/findmycells/interfaces.py#L466){target="_blank" style="float:right; font-size:smaller"}

### SettingsPage

>      SettingsPage (bundle_id:str,
>                    page_screen:traitlets.traitlets.MetaHasTraits,
>                    all_navigator_buttons:List, api:__main__.API)

Subclass of [`PageButtonBundle`](https://Defense-Circuits-Lab.github.io/findmycells/API documentation/interfaces.html#pagebuttonbundle) that implements the GUI interface that allows the user to specify all 
settings relevant to the findmycells project. It also enables saving & loading of the project status, 
and to browse through the file history that is automatically created by findmycells.

In [4]:
#| echo: false
#| output: asis
show_doc(ProcessingStepPage)

---

[source](https://github.com/Defense-Circuits-Lab/findmycells/blob/main/findmycells/interfaces.py#L778){target="_blank" style="float:right; font-size:smaller"}

### ProcessingStepPage

>      ProcessingStepPage (bundle_id:str,
>                          page_screen:traitlets.traitlets.MetaHasTraits,
>                          all_navigator_buttons:List, api:__main__.API)

Helper class that provides a standard way to create an ABC using
inheritance.

In [5]:
#| echo: false
#| output: asis
show_doc(InspectionPage)

---

[source](https://github.com/Defense-Circuits-Lab/findmycells/blob/main/findmycells/interfaces.py#L930){target="_blank" style="float:right; font-size:smaller"}

### InspectionPage

>      InspectionPage (bundle_id:str,
>                      page_screen:traitlets.traitlets.MetaHasTraits,
>                      all_navigator_buttons:List, api:__main__.API)

Helper class that provides a standard way to create an ABC using
inheritance.

In [6]:
#| echo: false
#| output: asis
show_doc(GUI)

---

[source](https://github.com/Defense-Circuits-Lab/findmycells/blob/main/findmycells/interfaces.py#L1109){target="_blank" style="float:right; font-size:smaller"}

### GUI

>      GUI ()

Initialize self.  See help(type(self)) for accurate signature.